In [ ]:
import numpy as np
import pandas as pd
pd.options.display.precision = 15
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import gc
warnings.filterwarnings('ignore')
import os
import openbabel
from tqdm import tqdm as tqdm
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.read_pickle('../input/train.pkl')
test = pd.read_pickle('../input/test.pkl')
structures = pd.read_pickle('../input/structures.pkl')
dist_each_other = pd.read_pickle('./feature_output/dist_each_other.pkl')

In [ ]:
def map_structure_info(df):
    df = pd.merge(df, structures, how = 'left',
                  left_on = ['molecule_name', f'atom_index_0'],
                  right_on = ['molecule_name',  'atom_index'],
                  suffixes=['_0', '_1'])
    
    df = df.drop('atom_index', axis=1)
    
    df = pd.merge(df, structures, how = 'left',
                  left_on = ['molecule_name', f'atom_index_1'],
                  right_on = ['molecule_name',  'atom_index'],
                  suffixes=['_0', '_1'])
    
    df = df.drop('atom_index', axis=1)
   
    return df


train = map_structure_info(train)
test = map_structure_info(test)

In [ ]:
def add_distance_features(df):
    df_p_0 = df[['x_0', 'y_0', 'z_0']].values
    df_p_1 = df[['x_1', 'y_1', 'z_1']].values
    
    df['dist'] = np.linalg.norm(df_p_0 - df_p_1, axis=1)

    return df


train = add_distance_features(train)
test = add_distance_features(test)

In [ ]:
atom_count = structures.groupby(['molecule_name', 'atom']).size().unstack(fill_value=0)
train = pd.merge(train, atom_count, how='left', on='molecule_name')
test = pd.merge(test, atom_count, how='left', on='molecule_name')

In [ ]:
obConversion = openbabel.OBConversion()
obConversion.SetInFormat("xyz")

structdir='../input/structures/'
mols=[]
mols_files=os.listdir(structdir)
mols_index=dict(map(reversed,enumerate(mols_files)))
for f in mols_index.keys():
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, structdir+f) 
    mols.append(mol)

In [ ]:
def Atoms(molname,AtomId1,AtomId2):
    mol=mols[mols_index[molname+'.xyz']]
    return mol, mol.GetAtomById(AtomId1), mol.GetAtomById(AtomId2)


def SecondAtom(bond,FirstAtom):
    if FirstAtom.GetId()==bond.GetBeginAtom().GetId(): return bond.GetEndAtom()
    else: return bond.GetBeginAtom()

    
def Angle2J(molname,AtomId1,AtomId2,debug=False):
    mol,firstAtom,lastAtom=Atoms(molname,AtomId1,AtomId2)
    if debug: print (mol.GetFormula())
    if debug: print(firstAtom.GetType(),firstAtom.GetId(),':',lastAtom.GetType(),lastAtom.GetId())
    for b in openbabel.OBAtomBondIter(firstAtom):
        secondAtom=SecondAtom(b,firstAtom)
        lastBond=secondAtom.GetBond(lastAtom)
        if lastBond:
            if debug: print('middle',secondAtom.GetId(),secondAtom.GetType())
            return firstAtom.GetAngle(secondAtom,lastAtom)

        
def Torsion3J(molname,AtomId1,AtomId2,debug=False):
    mol,firstAtom,lastAtom=Atoms(molname,AtomId1,AtomId2)
    if debug: print (molname, mol.GetFormula())
    if debug: print(firstAtom.GetType(),firstAtom.GetId(),':',lastAtom.GetType(),lastAtom.GetId())
    for b in openbabel.OBAtomBondIter(firstAtom):
        secondAtom=SecondAtom(b,firstAtom)
        for b2 in openbabel.OBAtomBondIter(secondAtom):
            thirdAtom=SecondAtom(b2,secondAtom)
            lastBond=thirdAtom.GetBond(lastAtom)
            if lastBond: # found!
                if debug: print(secondAtom.GetType(),secondAtom.GetId(),'<->',thirdAtom.GetType(),thirdAtom.GetId())
                return mol.GetTorsion(firstAtom,secondAtom,thirdAtom,lastAtom)
            
            
def add_angular_fetures(df):
    df['sp']=df.progress_apply(lambda row: Atoms(row.molecule_name, row.atom_index_0, row.atom_index_1)[2].GetHyb(),axis=1)
    df['Angle']=df.progress_apply(lambda row: Angle2J(row.molecule_name , row.atom_index_0, row.atom_index_1),axis=1)
    df['cosA']=np.cos(np.deg2rad(df['Angle']))
    df['cos2A']=np.cos(2*np.deg2rad(df['Angle']))
    df['Torsion']=df.progress_apply(lambda row: Torsion3J(row.molecule_name , row.atom_index_0, row.atom_index_1),axis=1) 
    df['cosT']=np.cos(np.deg2rad(df['Torsion']))
    df['cos2T']=np.cos(2*np.deg2rad(df['Torsion']))
    
    return df


train = add_angular_fetures(train)
test = add_angular_fetures(test)

In [ ]:
def map_dist_each_other(df):
    df = pd.merge(df, dist_each_other, how = 'left',
                  left_on = ['molecule_name', f'atom_index_0'],
                  right_on = ['molecule_name',  'atom_index'],
                  suffixes=['_atom_0', '_atom_1'])
    
    df = df.drop('atom_index', axis=1)
    
    df = pd.merge(df, dist_each_other, how = 'left',
                  left_on = ['molecule_name', f'atom_index_1'],
                  right_on = ['molecule_name',  'atom_index'],
                  suffixes=['_atom_0', '_atom_1'])
    
    df = df.drop('atom_index', axis=1)
    
    return df


train = map_dist_each_other(train)
test = map_dist_each_other(test)

In [ ]:
def add_dist_features(df):
    atoms = ['H', 'C', 'N', 'O', 'F']

    for i in [0, 1]:
        each_atom_idx = pd.DataFrame()
        
        for atom in atoms:
            each_atom = pd.DataFrame()
            
            for j in range(20):
                each_atom[f'dist_{atom}_{j}_atom_{i}'] = df[f'dist_{atom}_{j}_atom_{i}']
            
            each_atom_mean = each_atom.mean(axis=1, skipna=True)
            df[f'dist_{atom}_atom_{i}'] = each_atom_mean
            each_atom_idx[f'dist_{atom}_atom_{i}'] = each_atom_mean
            
        df[f'Dist_mean_atom_{i}'] = each_atom_idx.mean(axis=1, skipna=True)
        
    return df


train = add_dist_features(train)
test = add_dist_features(test)

In [ ]:
# def dist_to_potential(df):
#     atoms = ['H', 'C', 'N', 'O', 'F']
#     cols = df.columns
    
#     for i in [0, 1]:
#         for atom in atoms:
#             for j in range(20):
#                 df[f'potential_{atom}_{j}_atom_{i}'] = 1 / df[f'dist_{atom}_{j}_atom_{i}']
                    
#     return df

# train = dist_to_potential(train)
# test = dist_to_potential(test)

In [ ]:
def add_potential_features(df):
    atoms = ['H', 'C', 'N', 'O', 'F']

    for i in [0, 1]:
        each_atom_idx = np.zeros(len(df))
        for atom in atoms:
            each_atom = np.zeros(len(df))
            for j in range(20):
                potential = (1 / df[f'dist_{atom}_{j}_atom_{i}']).fillna(0)
                each_atom += potential.values
            
            each_atom_idx += each_atom
            df[f'coulomb_{atom}_atom_{i}'] = each_atom
        df[f'Coulomb_atom_{i}'] = each_atom_idx
         
    for i in [0, 1]:
        each_atom_idx = np.zeros(len(df))
        for atom in atoms:
            each_atom = np.zeros(len(df))
            for j in range(20):
                potential = (1 / (df[f'dist_{atom}_{j}_atom_{i}'] ** 3)).fillna(0)
                each_atom += potential.values
            
            each_atom_idx += each_atom
            df[f'dipole_{atom}_atom_{i}'] = each_atom
        df[f'Dipole_atom_{i}'] = each_atom_idx
        
    for i in [0, 1]:
        each_atom_idx = np.zeros(len(df))
        for atom in atoms:
            each_atom = np.zeros(len(df))
            for j in range(20):
                potential = (1 / (df[f'dist_{atom}_{j}_atom_{i}'] ** 6)).fillna(0)
                each_atom += potential.values
            
            each_atom_idx += each_atom
            df[f'vander_{atom}_atom_{i}'] = each_atom
        df[f'Vander_atom_{i}'] = each_atom_idx
        
    for i in [0, 1]:
        each_atom_idx = np.zeros(len(df))
        for atom in atoms:
            each_atom = np.zeros(len(df))
            for j in range(20):
                potential = (np.exp(-df[f'dist_{atom}_{j}_atom_{i}']) / df[f'dist_{atom}_{j}_atom_{i}']).fillna(0)
                each_atom += potential.values
            
            each_atom_idx += each_atom
            df[f'yukawa_{atom}_atom_{i}'] = each_atom
        df[f'Yukawa_atom_{i}'] = each_atom_idx
        
    df['All_potential_0'] = df['Coulomb_atom_0'] + df['Dipole_atom_0'] + df['Vander_atom_0'] + df['Yukawa_atom_0']
    df['All_potential_1'] = df['Coulomb_atom_1'] + df['Dipole_atom_1'] + df['Vander_atom_1'] + df['Yukawa_atom_1']
    
    return df


train = add_potential_features(train)
test = add_potential_features(test)

In [ ]:
def dist_zero_to_nan(df):
    atoms = ['H', 'C', 'N', 'O', 'F']
    for i in [0, 1]:
        for atom in atoms:            
            df[f'dist_{atom}_atom_{i}'] = df[f'dist_{atom}_atom_{i}'].replace(0, np.nan)
        
    return df


train = dist_zero_to_nan(train)
test = dist_zero_to_nan(test)

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
train.to_pickle('./feature_output/train_merged.pkl')
test.to_pickle('./feature_output/test_merged.pkl')

In [ ]:
types = train['type'].unique()

for t in types:
    train_idx = train['type'] == t
    test_idx = test['type'] == t
    
    train_type = train.loc[train_idx]
    test_type = test.loc[test_idx]
    
    train_type.to_csv(f'./feature_output/each_type/train_{t}.csv', index=False)
    test_type.to_csv(f'./feature_output/each_type/test_{t}.csv', index=False)

In [ ]:
print(train.shape, test.shape)

In [ ]:
train.head()

In [ ]:
test.head()